In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature = 0.1,)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

In [2]:
examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You should be able to express all the movies with 3 emoticons."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{movie}")
    ]
)


def load_history(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_history)| final_prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context({"input": movie}, {"output": result.content})
    print(result.content)
    print(load_history(""))

In [3]:
invoke_chain("Avengers")

🦸‍♂️💥🌍
[HumanMessage(content='Avengers'), AIMessage(content='🦸\u200d♂️💥🌍')]


In [4]:
invoke_chain("Avatar")

🪐🌿💙
[HumanMessage(content='Avengers'), AIMessage(content='🦸\u200d♂️💥🌍'), HumanMessage(content='Avatar'), AIMessage(content='🪐🌿💙')]


In [5]:
invoke_chain("What's the movie I asked you first?")

Avengers
[HumanMessage(content='Avengers'), AIMessage(content='🦸\u200d♂️💥🌍'), HumanMessage(content='Avatar'), AIMessage(content='🪐🌿💙'), HumanMessage(content="What's the movie I asked you first?"), AIMessage(content='Avengers')]
